Importando Librerías

In [94]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, DoubleType
from pyspark.sql.types import *
from pyspark.sql.functions import col, when, lit, avg, sum, max
from datetime import datetime, timedelta

Creando Sesión

In [3]:
spark = SparkSession.builder.getOrCreate()

In [3]:
spark

In [5]:
data2 = [("James","","Smith","36636","M",3000),
    ("Michael","Rose","","40288","M",4000),
    ("Robert","","Williams","42114","M",4000),
    ("Maria","Anne","Jones","39192","F",4000),
    ("Jen","Mary","Brown","","F",-1)
  ]

schema = StructType([ \
    StructField("firstname",StringType(),True), \
    StructField("middlename",StringType(),True), \
    StructField("lastname",StringType(),True), \
    StructField("id", StringType(), True), \
    StructField("gender", StringType(), True), \
    StructField("salary", IntegerType(), True) \
  ])
 
df = spark.createDataFrame(data=data2,schema=schema)
df.printSchema()
df.show(truncate=False)

root
 |-- firstname: string (nullable = true)
 |-- middlename: string (nullable = true)
 |-- lastname: string (nullable = true)
 |-- id: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: integer (nullable = true)

+---------+----------+--------+-----+------+------+
|firstname|middlename|lastname|id   |gender|salary|
+---------+----------+--------+-----+------+------+
|James    |          |Smith   |36636|M     |3000  |
|Michael  |Rose      |        |40288|M     |4000  |
|Robert   |          |Williams|42114|M     |4000  |
|Maria    |Anne      |Jones   |39192|F     |4000  |
|Jen      |Mary      |Brown   |     |F     |-1    |
+---------+----------+--------+-----+------+------+



In [6]:
ruta = 'gs://dmc_dde_barb/archivos/persona.data'

df_schema = StructType([
StructField("ID", StringType(),True),
StructField("NOMBRE", StringType(),True),
StructField("TELEFONO", StringType(),True),
StructField("CORREO", StringType(),True),
StructField("FECHA_INGRESO", StringType(),True),
StructField("EDAD", IntegerType(),True),
StructField("SALARIO", DoubleType(),True),
StructField("ID_EMPRESA", StringType(),True),
])


df_with_schema = spark.read.format("CSV").option("header","true").option("delimiter","|").schema(df_schema).load(ruta)

In [7]:
df_with_schema.show(10)

+---+---------+--------------+--------------------+-------------+----+-------+----------+
| ID|   NOMBRE|      TELEFONO|              CORREO|FECHA_INGRESO|EDAD|SALARIO|ID_EMPRESA|
+---+---------+--------------+--------------------+-------------+----+-------+----------+
|  1|     Carl|1-745-633-9145|arcu.Sed.et@ante....|   2004-04-23|  32|20095.0|         5|
|  2|Priscilla|      155-2498|Donec.egestas.Ali...|   2019-02-17|  34| 9298.0|         2|
|  3|  Jocelyn|1-204-956-8594|amet.diam@loborti...|   2002-08-01|  27|10853.0|         3|
|  4|    Aidan|1-719-862-9385|euismod.et.commod...|   2018-11-06|  29| 3387.0|        10|
|  5|  Leandra|      839-8044|at@pretiumetrutru...|   2002-10-10|  41|22102.0|         1|
|  6|     Bert|      797-4453|a.felis.ullamcorp...|   2017-04-25|  70| 7800.0|         7|
|  7|     Mark|1-680-102-6792|Quisque.ac@placer...|   2006-04-21|  52| 8112.0|         5|
|  8|    Jonah|      214-2975|eu.ultrices.sit@v...|   2017-10-07|  23|17040.0|         5|
|  9|    H

In [8]:
df_with_schema.printSchema()

root
 |-- ID: string (nullable = true)
 |-- NOMBRE: string (nullable = true)
 |-- TELEFONO: string (nullable = true)
 |-- CORREO: string (nullable = true)
 |-- FECHA_INGRESO: string (nullable = true)
 |-- EDAD: integer (nullable = true)
 |-- SALARIO: double (nullable = true)
 |-- ID_EMPRESA: string (nullable = true)



In [14]:
df_with_schema.select("nombre","fecha_ingreso").show(4)

+---------+-------------+
|   nombre|fecha_ingreso|
+---------+-------------+
|     Carl|   2004-04-23|
|Priscilla|   2019-02-17|
|  Jocelyn|   2002-08-01|
|    Aidan|   2018-11-06|
+---------+-------------+
only showing top 4 rows



In [13]:
df_with_schema.select(df_with_schema.NOMBRE, df_with_schema.FECHA_INGRESO).show(4)

+---------+-------------+
|   NOMBRE|FECHA_INGRESO|
+---------+-------------+
|     Carl|   2004-04-23|
|Priscilla|   2019-02-17|
|  Jocelyn|   2002-08-01|
|    Aidan|   2018-11-06|
+---------+-------------+
only showing top 4 rows



In [16]:
df_with_schema.select(df_with_schema["nombre"],df_with_schema["fecha_ingreso"]).show(5)

+---------+-------------+
|   nombre|fecha_ingreso|
+---------+-------------+
|     Carl|   2004-04-23|
|Priscilla|   2019-02-17|
|  Jocelyn|   2002-08-01|
|    Aidan|   2018-11-06|
|  Leandra|   2002-10-10|
+---------+-------------+
only showing top 5 rows



In [19]:
df_with_schema.select(col("nombre"),col("fecha_ingreso")).show(4)

+---------+-------------+
|   nombre|fecha_ingreso|
+---------+-------------+
|     Carl|   2004-04-23|
|Priscilla|   2019-02-17|
|  Jocelyn|   2002-08-01|
|    Aidan|   2018-11-06|
+---------+-------------+
only showing top 4 rows



In [27]:
df_with_schema.select(df_with_schema.colRegex("`^.*re*.$`")).show(5)

+---------+--------------------+
|   NOMBRE|              CORREO|
+---------+--------------------+
|     Carl|arcu.Sed.et@ante....|
|Priscilla|Donec.egestas.Ali...|
|  Jocelyn|amet.diam@loborti...|
|    Aidan|euismod.et.commod...|
|  Leandra|at@pretiumetrutru...|
+---------+--------------------+
only showing top 5 rows



In [24]:
df_with_schema.withColumn("salario",col("salario").cast("integer")).show(5)

+---+---------+--------------+--------------------+-------------+----+-------+----------+
| ID|   NOMBRE|      TELEFONO|              CORREO|FECHA_INGRESO|EDAD|salario|ID_EMPRESA|
+---+---------+--------------+--------------------+-------------+----+-------+----------+
|  1|     Carl|1-745-633-9145|arcu.Sed.et@ante....|   2004-04-23|  32|  20095|         5|
|  2|Priscilla|      155-2498|Donec.egestas.Ali...|   2019-02-17|  34|   9298|         2|
|  3|  Jocelyn|1-204-956-8594|amet.diam@loborti...|   2002-08-01|  27|  10853|         3|
|  4|    Aidan|1-719-862-9385|euismod.et.commod...|   2018-11-06|  29|   3387|        10|
|  5|  Leandra|      839-8044|at@pretiumetrutru...|   2002-10-10|  41|  22102|         1|
+---+---------+--------------+--------------------+-------------+----+-------+----------+
only showing top 5 rows



In [30]:
df_with_schema.withColumn("incremento",col("salario")*0.10).select(col("nombre"),col("salario"),col("incremento")).show(5)

+---------+-------+------------------+
|   nombre|salario|        incremento|
+---------+-------+------------------+
|     Carl|20095.0|            2009.5|
|Priscilla| 9298.0| 929.8000000000001|
|  Jocelyn|10853.0|            1085.3|
|    Aidan| 3387.0|338.70000000000005|
|  Leandra|22102.0|2210.2000000000003|
+---------+-------+------------------+
only showing top 5 rows



In [42]:
limite_salario_bajo = 5000
inicio_salario_alto = 30000

df_with_schema.withColumn("tipo_salario", when(col("salario")<limite_salario_bajo,"salario bajo")\
                                              .when(((col("salario")>=limite_salario_bajo) & (col("salario")<inicio_salario_alto)),"salario medio")\
                                              .otherwise("salario alto")).show(10)

+---+---------+--------------+--------------------+-------------+----+-------+----------+-------------+
| ID|   NOMBRE|      TELEFONO|              CORREO|FECHA_INGRESO|EDAD|SALARIO|ID_EMPRESA| tipo_salario|
+---+---------+--------------+--------------------+-------------+----+-------+----------+-------------+
|  1|     Carl|1-745-633-9145|arcu.Sed.et@ante....|   2004-04-23|  32|20095.0|         5|salario medio|
|  2|Priscilla|      155-2498|Donec.egestas.Ali...|   2019-02-17|  34| 9298.0|         2|salario medio|
|  3|  Jocelyn|1-204-956-8594|amet.diam@loborti...|   2002-08-01|  27|10853.0|         3|salario medio|
|  4|    Aidan|1-719-862-9385|euismod.et.commod...|   2018-11-06|  29| 3387.0|        10| salario bajo|
|  5|  Leandra|      839-8044|at@pretiumetrutru...|   2002-10-10|  41|22102.0|         1|salario medio|
|  6|     Bert|      797-4453|a.felis.ullamcorp...|   2017-04-25|  70| 7800.0|         7|salario medio|
|  7|     Mark|1-680-102-6792|Quisque.ac@placer...|   2006-04-21

In [43]:
df_with_schema.withColumnRenamed("salario","salary").show(10)

+---+---------+--------------+--------------------+-------------+----+-------+----------+
| ID|   NOMBRE|      TELEFONO|              CORREO|FECHA_INGRESO|EDAD| salary|ID_EMPRESA|
+---+---------+--------------+--------------------+-------------+----+-------+----------+
|  1|     Carl|1-745-633-9145|arcu.Sed.et@ante....|   2004-04-23|  32|20095.0|         5|
|  2|Priscilla|      155-2498|Donec.egestas.Ali...|   2019-02-17|  34| 9298.0|         2|
|  3|  Jocelyn|1-204-956-8594|amet.diam@loborti...|   2002-08-01|  27|10853.0|         3|
|  4|    Aidan|1-719-862-9385|euismod.et.commod...|   2018-11-06|  29| 3387.0|        10|
|  5|  Leandra|      839-8044|at@pretiumetrutru...|   2002-10-10|  41|22102.0|         1|
|  6|     Bert|      797-4453|a.felis.ullamcorp...|   2017-04-25|  70| 7800.0|         7|
|  7|     Mark|1-680-102-6792|Quisque.ac@placer...|   2006-04-21|  52| 8112.0|         5|
|  8|    Jonah|      214-2975|eu.ultrices.sit@v...|   2017-10-07|  23|17040.0|         5|
|  9|    H

In [44]:
df_with_schema.drop("salario").show(5)

+---+---------+--------------+--------------------+-------------+----+----------+
| ID|   NOMBRE|      TELEFONO|              CORREO|FECHA_INGRESO|EDAD|ID_EMPRESA|
+---+---------+--------------+--------------------+-------------+----+----------+
|  1|     Carl|1-745-633-9145|arcu.Sed.et@ante....|   2004-04-23|  32|         5|
|  2|Priscilla|      155-2498|Donec.egestas.Ali...|   2019-02-17|  34|         2|
|  3|  Jocelyn|1-204-956-8594|amet.diam@loborti...|   2002-08-01|  27|         3|
|  4|    Aidan|1-719-862-9385|euismod.et.commod...|   2018-11-06|  29|        10|
|  5|  Leandra|      839-8044|at@pretiumetrutru...|   2002-10-10|  41|         1|
+---+---------+--------------+--------------------+-------------+----+----------+
only showing top 5 rows



In [46]:
df_with_schema.withColumn("periodo",lit("202503")).show(10)

+---+---------+--------------+--------------------+-------------+----+-------+----------+-------+
| ID|   NOMBRE|      TELEFONO|              CORREO|FECHA_INGRESO|EDAD|SALARIO|ID_EMPRESA|periodo|
+---+---------+--------------+--------------------+-------------+----+-------+----------+-------+
|  1|     Carl|1-745-633-9145|arcu.Sed.et@ante....|   2004-04-23|  32|20095.0|         5| 202503|
|  2|Priscilla|      155-2498|Donec.egestas.Ali...|   2019-02-17|  34| 9298.0|         2| 202503|
|  3|  Jocelyn|1-204-956-8594|amet.diam@loborti...|   2002-08-01|  27|10853.0|         3| 202503|
|  4|    Aidan|1-719-862-9385|euismod.et.commod...|   2018-11-06|  29| 3387.0|        10| 202503|
|  5|  Leandra|      839-8044|at@pretiumetrutru...|   2002-10-10|  41|22102.0|         1| 202503|
|  6|     Bert|      797-4453|a.felis.ullamcorp...|   2017-04-25|  70| 7800.0|         7| 202503|
|  7|     Mark|1-680-102-6792|Quisque.ac@placer...|   2006-04-21|  52| 8112.0|         5| 202503|
|  8|    Jonah|     

In [52]:
fecha_actual = datetime.now()
print("hora del servidor: ",fecha_actual)
fecha_peru = fecha_actual - timedelta(hours=5)
periodo = fecha_peru.strftime("%Y%m")
print(periodo)

hora del servidor:  2025-03-04 02:32:30.982753
202503


In [54]:
df_with_schema.filter(col("id_empresa")==5).show(10)

+---+------+--------------+--------------------+-------------+----+-------+----------+
| ID|NOMBRE|      TELEFONO|              CORREO|FECHA_INGRESO|EDAD|SALARIO|ID_EMPRESA|
+---+------+--------------+--------------------+-------------+----+-------+----------+
|  1|  Carl|1-745-633-9145|arcu.Sed.et@ante....|   2004-04-23|  32|20095.0|         5|
|  7|  Mark|1-680-102-6792|Quisque.ac@placer...|   2006-04-21|  52| 8112.0|         5|
|  8| Jonah|      214-2975|eu.ultrices.sit@v...|   2017-10-07|  23|17040.0|         5|
| 13|Trevor|      512-1955|Nunc.quis.arcu@eg...|   2010-08-06|  34| 9501.0|         5|
| 15| Wanda|      359-6973|Nam.nulla.magna@I...|   2005-08-21|  27| 1539.0|         5|
| 35|Aurora|1-865-751-3479|      magna@Cras.net|   2017-10-21|  54| 4588.0|         5|
| 50|  Ross|1-587-285-1837|at.risus@milacini...|   2009-11-03|  31|19092.0|         5|
| 51| Damon|      368-7630|nunc@dapibusquamq...|   2016-08-11|  49| 2669.0|         5|
| 59|Quemby|      930-5882|lorem.ut.aliquam

In [56]:
df_with_schema.filter((col("id_empresa")==5) & (col("salario")>=10000)).show()

+---+------+--------------+--------------------+-------------+----+-------+----------+
| ID|NOMBRE|      TELEFONO|              CORREO|FECHA_INGRESO|EDAD|SALARIO|ID_EMPRESA|
+---+------+--------------+--------------------+-------------+----+-------+----------+
|  1|  Carl|1-745-633-9145|arcu.Sed.et@ante....|   2004-04-23|  32|20095.0|         5|
|  8| Jonah|      214-2975|eu.ultrices.sit@v...|   2017-10-07|  23|17040.0|         5|
| 50|  Ross|1-587-285-1837|at.risus@milacini...|   2009-11-03|  31|19092.0|         5|
| 59|Quemby|      930-5882|lorem.ut.aliquam@...|   2017-10-04|  26|12092.0|         5|
| 86|  Jack|      860-9554|parturient.montes...|   2017-03-10|  58|14473.0|         5|
+---+------+--------------+--------------------+-------------+----+-------+----------+



In [63]:
lista_empresas = [2,3,5]
df_with_schema.filter(col("id_empresa").isin([2,3,5])).show(3)
df_with_schema.filter(col("id_empresa").isin(2,3,5)).show(3)
df_with_schema.filter(col("id_empresa").isin(lista_empresas)).show(3)


+---+---------+--------------+--------------------+-------------+----+-------+----------+
| ID|   NOMBRE|      TELEFONO|              CORREO|FECHA_INGRESO|EDAD|SALARIO|ID_EMPRESA|
+---+---------+--------------+--------------------+-------------+----+-------+----------+
|  1|     Carl|1-745-633-9145|arcu.Sed.et@ante....|   2004-04-23|  32|20095.0|         5|
|  2|Priscilla|      155-2498|Donec.egestas.Ali...|   2019-02-17|  34| 9298.0|         2|
|  3|  Jocelyn|1-204-956-8594|amet.diam@loborti...|   2002-08-01|  27|10853.0|         3|
+---+---------+--------------+--------------------+-------------+----+-------+----------+
only showing top 3 rows

+---+---------+--------------+--------------------+-------------+----+-------+----------+
| ID|   NOMBRE|      TELEFONO|              CORREO|FECHA_INGRESO|EDAD|SALARIO|ID_EMPRESA|
+---+---------+--------------+--------------------+-------------+----+-------+----------+
|  1|     Carl|1-745-633-9145|arcu.Sed.et@ante....|   2004-04-23|  32|20095

In [64]:
df_with_schema.filter(col("nombre").like("%ar%")).show()

+---+-------+--------------+--------------------+-------------+----+-------+----------+
| ID| NOMBRE|      TELEFONO|              CORREO|FECHA_INGRESO|EDAD|SALARIO|ID_EMPRESA|
+---+-------+--------------+--------------------+-------------+----+-------+----------+
|  1|   Carl|1-745-633-9145|arcu.Sed.et@ante....|   2004-04-23|  32|20095.0|         5|
|  7|   Mark|1-680-102-6792|Quisque.ac@placer...|   2006-04-21|  52| 8112.0|         5|
| 17|   Omar|      720-1543|Phasellus.vitae.m...|   2014-06-24|  60| 6851.0|         6|
| 21|Carissa|1-300-877-0859|dignissim.pharetr...|   2011-10-16|  31| 1952.0|        10|
| 25|  Pearl|1-850-202-3373|vel.convallis@rho...|   2018-12-21|  52|14756.0|         6|
| 39|Carolyn|      846-7060|metus.Aenean.sed@...|   2013-05-29|  64|22838.0|         6|
| 54|   Lars|1-554-600-0855|     commodo@Nam.edu|   2005-06-22|  25|20573.0|         1|
| 60|Bernard|      492-8823|vel.faucibus@Done...|   2005-04-15|  27|10825.0|         2|
| 76|   Omar|1-325-245-9578|elit

In [65]:
data = [("James", "Sales", 3000), \
    ("Michael", "Sales", 4600), \
    ("Robert", "Sales", 4100), \
    ("Maria", "Finance", 3000), \
    ("James", "Sales", 3000), \
    ("Scott", "Finance", 3300), \
    ("Jen", "Finance", 3900), \
    ("Jeff", "Marketing", 3000), \
    ("Kumar", "Marketing", 2000), \
    ("Saif", "Sales", 4100) \
  ]
columns= ["employee_name", "department", "salary"]
df_a = spark.createDataFrame(data = data, schema = columns)
df_a.printSchema()
df_a.show(truncate=False)

root
 |-- employee_name: string (nullable = true)
 |-- department: string (nullable = true)
 |-- salary: long (nullable = true)

+-------------+----------+------+
|employee_name|department|salary|
+-------------+----------+------+
|James        |Sales     |3000  |
|Michael      |Sales     |4600  |
|Robert       |Sales     |4100  |
|Maria        |Finance   |3000  |
|James        |Sales     |3000  |
|Scott        |Finance   |3300  |
|Jen          |Finance   |3900  |
|Jeff         |Marketing |3000  |
|Kumar        |Marketing |2000  |
|Saif         |Sales     |4100  |
+-------------+----------+------+



In [70]:
df_a.count()

10

In [72]:
df_a.distinct().show()
df_a.distinct().count()

+-------------+----------+------+
|employee_name|department|salary|
+-------------+----------+------+
|        James|     Sales|  3000|
|       Robert|     Sales|  4100|
|        Maria|   Finance|  3000|
|      Michael|     Sales|  4600|
|         Saif|     Sales|  4100|
|        Scott|   Finance|  3300|
|         Jeff| Marketing|  3000|
|          Jen|   Finance|  3900|
|        Kumar| Marketing|  2000|
+-------------+----------+------+



9

In [74]:
df_a.dropDuplicates().show()

+-------------+----------+------+
|employee_name|department|salary|
+-------------+----------+------+
|        James|     Sales|  3000|
|       Robert|     Sales|  4100|
|        Maria|   Finance|  3000|
|      Michael|     Sales|  4600|
|         Saif|     Sales|  4100|
|        Scott|   Finance|  3300|
|         Jeff| Marketing|  3000|
|          Jen|   Finance|  3900|
|        Kumar| Marketing|  2000|
+-------------+----------+------+



In [75]:
df_a.dropDuplicates(["department","salary"]).show()

+-------------+----------+------+
|employee_name|department|salary|
+-------------+----------+------+
|        Maria|   Finance|  3000|
|        Scott|   Finance|  3300|
|          Jen|   Finance|  3900|
|        Kumar| Marketing|  2000|
|         Jeff| Marketing|  3000|
|        James|     Sales|  3000|
|       Robert|     Sales|  4100|
|      Michael|     Sales|  4600|
+-------------+----------+------+



In [76]:
df_a.sort(col("employee_name")).show()

+-------------+----------+------+
|employee_name|department|salary|
+-------------+----------+------+
|        James|     Sales|  3000|
|        James|     Sales|  3000|
|         Jeff| Marketing|  3000|
|          Jen|   Finance|  3900|
|        Kumar| Marketing|  2000|
|        Maria|   Finance|  3000|
|      Michael|     Sales|  4600|
|       Robert|     Sales|  4100|
|         Saif|     Sales|  4100|
|        Scott|   Finance|  3300|
+-------------+----------+------+



In [77]:
df_with_schema.show()

+---+---------+--------------+--------------------+-------------+----+-------+----------+
| ID|   NOMBRE|      TELEFONO|              CORREO|FECHA_INGRESO|EDAD|SALARIO|ID_EMPRESA|
+---+---------+--------------+--------------------+-------------+----+-------+----------+
|  1|     Carl|1-745-633-9145|arcu.Sed.et@ante....|   2004-04-23|  32|20095.0|         5|
|  2|Priscilla|      155-2498|Donec.egestas.Ali...|   2019-02-17|  34| 9298.0|         2|
|  3|  Jocelyn|1-204-956-8594|amet.diam@loborti...|   2002-08-01|  27|10853.0|         3|
|  4|    Aidan|1-719-862-9385|euismod.et.commod...|   2018-11-06|  29| 3387.0|        10|
|  5|  Leandra|      839-8044|at@pretiumetrutru...|   2002-10-10|  41|22102.0|         1|
|  6|     Bert|      797-4453|a.felis.ullamcorp...|   2017-04-25|  70| 7800.0|         7|
|  7|     Mark|1-680-102-6792|Quisque.ac@placer...|   2006-04-21|  52| 8112.0|         5|
|  8|    Jonah|      214-2975|eu.ultrices.sit@v...|   2017-10-07|  23|17040.0|         5|
|  9|    H

In [81]:
df_with_schema.sort(col("id_empresa").cast("integer")).show(30)

+---+---------+--------------+--------------------+-------------+----+-------+----------+
| ID|   NOMBRE|      TELEFONO|              CORREO|FECHA_INGRESO|EDAD|SALARIO|ID_EMPRESA|
+---+---------+--------------+--------------------+-------------+----+-------+----------+
| 32|   Gisela|      406-8031|Praesent.luctus@d...|   2002-08-21|  67| 6497.0|         1|
|  5|  Leandra|      839-8044|at@pretiumetrutru...|   2002-10-10|  41|22102.0|         1|
| 54|     Lars|1-554-600-0855|     commodo@Nam.edu|   2005-06-22|  25|20573.0|         1|
| 70|     Suki|1-654-418-8537|varius.orci.in@co...|   2010-05-15|  43|12029.0|         1|
| 87|    Karly|1-644-725-7241|tempor.erat@feugi...|   2011-06-12|  25| 3715.0|         1|
| 93|   Althea|1-163-702-1244|sit.amet.nulla@el...|   2002-09-01|  24| 8818.0|         1|
| 99|      Ray|1-420-314-2886|ac.risus.Morbi@te...|   2011-12-30|  26| 5570.0|         1|
|  2|Priscilla|      155-2498|Donec.egestas.Ali...|   2019-02-17|  34| 9298.0|         2|
| 14|    A

In [82]:
df_with_schema.orderBy(col("id_empresa").cast("integer")).show(30)

+---+---------+--------------+--------------------+-------------+----+-------+----------+
| ID|   NOMBRE|      TELEFONO|              CORREO|FECHA_INGRESO|EDAD|SALARIO|ID_EMPRESA|
+---+---------+--------------+--------------------+-------------+----+-------+----------+
| 32|   Gisela|      406-8031|Praesent.luctus@d...|   2002-08-21|  67| 6497.0|         1|
|  5|  Leandra|      839-8044|at@pretiumetrutru...|   2002-10-10|  41|22102.0|         1|
| 54|     Lars|1-554-600-0855|     commodo@Nam.edu|   2005-06-22|  25|20573.0|         1|
| 70|     Suki|1-654-418-8537|varius.orci.in@co...|   2010-05-15|  43|12029.0|         1|
| 87|    Karly|1-644-725-7241|tempor.erat@feugi...|   2011-06-12|  25| 3715.0|         1|
| 93|   Althea|1-163-702-1244|sit.amet.nulla@el...|   2002-09-01|  24| 8818.0|         1|
| 99|      Ray|1-420-314-2886|ac.risus.Morbi@te...|   2011-12-30|  26| 5570.0|         1|
|  2|Priscilla|      155-2498|Donec.egestas.Ali...|   2019-02-17|  34| 9298.0|         2|
| 14|    A

In [83]:
df_with_schema.orderBy(col("id_empresa").cast("integer").asc(),col("edad").desc()).show(30)

+---+---------+--------------+--------------------+-------------+----+-------+----------+
| ID|   NOMBRE|      TELEFONO|              CORREO|FECHA_INGRESO|EDAD|SALARIO|ID_EMPRESA|
+---+---------+--------------+--------------------+-------------+----+-------+----------+
| 32|   Gisela|      406-8031|Praesent.luctus@d...|   2002-08-21|  67| 6497.0|         1|
| 70|     Suki|1-654-418-8537|varius.orci.in@co...|   2010-05-15|  43|12029.0|         1|
|  5|  Leandra|      839-8044|at@pretiumetrutru...|   2002-10-10|  41|22102.0|         1|
| 99|      Ray|1-420-314-2886|ac.risus.Morbi@te...|   2011-12-30|  26| 5570.0|         1|
| 54|     Lars|1-554-600-0855|     commodo@Nam.edu|   2005-06-22|  25|20573.0|         1|
| 87|    Karly|1-644-725-7241|tempor.erat@feugi...|   2011-06-12|  25| 3715.0|         1|
| 93|   Althea|1-163-702-1244|sit.amet.nulla@el...|   2002-09-01|  24| 8818.0|         1|
| 43|    Yetta|      986-0220|vitae@dapibusrutr...|   2008-03-24|  61|21452.0|         2|
| 14|    A

In [95]:
df_with_schema.groupBy("id_empresa")\
    .agg(sum("salario").alias("planilla"),\
         avg("edad").alias("promedio_edad"),\
         max("salario").alias("salario_maximo")
        ).show(truncate=False)

         

+----------+--------+------------------+--------------+
|id_empresa|planilla|promedio_edad     |salario_maximo|
+----------+--------+------------------+--------------+
|7         |106710.0|34.55555555555556 |21556.0       |
|9         |91678.0 |37.666666666666664|23051.0       |
|3         |151700.0|39.63636363636363 |23820.0       |
|6         |135243.0|50.0              |22838.0       |
|1         |79304.0 |35.857142857142854|22102.0       |
|8         |73319.0 |39.888888888888886|19943.0       |
|5         |136609.0|41.214285714285715|20095.0       |
|10        |82012.0 |40.888888888888886|24575.0       |
|4         |155503.0|38.875            |24305.0       |
|2         |156377.0|39.785714285714285|22953.0       |
+----------+--------+------------------+--------------+



In [97]:
df_with_schema.groupBy("id_empresa")\
    .agg(sum("salario").alias("planilla"),\
         avg("edad").alias("promedio_edad"),\
         max("salario").alias("salario_maximo")
        ).where(col("planilla")>=100000).show(10)

+----------+--------+------------------+--------------+
|id_empresa|planilla|     promedio_edad|salario_maximo|
+----------+--------+------------------+--------------+
|         7|106710.0| 34.55555555555556|       21556.0|
|         3|151700.0| 39.63636363636363|       23820.0|
|         6|135243.0|              50.0|       22838.0|
|         5|136609.0|41.214285714285715|       20095.0|
|         4|155503.0|            38.875|       24305.0|
|         2|156377.0|39.785714285714285|       22953.0|
+----------+--------+------------------+--------------+



In [99]:
#PARTITIONBY | REPARTITION | COALESCE
#PARTITIONBY->FUNCIONAL MOMENTO DE ESCRIBIR EN DISCO - PARTICIONANDO DE DATOS EN DISCO df.writhe.partitionby("campo_pariticion")
#REPARTITION -> aumentar o disminuir particiones en memoria df.repartition(4)
#COALSESCE -> disminuir u optimizar particiones en memoria. df.coalesce()

ruta_destino = 'gs://dmc_dde_barb/archivos/salida/'
df_with_schema.write.mode("overwrite").partitionBy("id_empresa").format("parquet").save(ruta_destino)

#ruta_guardado = 'gs://dmc_datalake_dde_11_jmsp/archivos/persona_output/'
#df.write.mode("overwrite").partitionBy("ID_EMPRESA").format("parquet").save(ruta_guardado)
